In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans, AgglomerativeClustering, OPTICS, DBSCAN, cluster_optics_dbscan
from sklearn_extra.cluster import KMedoids
import random
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.base import clone
import yfinance as yf
from utils.utils import Portfolio
from utils.helpers import pooled_within_ssd, gen_realizations, gap_statistic, cluster_range, plot_internal, plot_internal_zoom_range, calcCorr, correlDist
from utils.clusters import kmeans_cluster, kmedoids_cluster, agglomerative_cluster

import tslearn
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler, TimeSeriesScalerMinMax
from tslearn.piecewise import PiecewiseAggregateApproximation, SymbolicAggregateApproximation, OneD_SymbolicAggregateApproximation
from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import cdist_dtw

### Load Data from YF for S&P500 and save it

In [3]:
sp500 = Portfolio('2013-01-01', '2024-01-01')
sp500_train = Portfolio('2013-01-01', '2023-01-01')
sp500_test = Portfolio('2023-01-01', '2024-01-01')
dji_test = Portfolio('2023-01-01','2024-01-01', file = 'data/dji.csv')

## Baselines for Sharpe Ratio: DJI & 30 Random

In [4]:
baseline = dji_test.metrics.copy()
baseline.loc['SP500'] = [sp500_test.get_portf_sharpe(), sp500_test.get_portf_sortino(), sp500_test.get_portf_return()]
baseline

,Sharpe,Sortino,Returns
DJI,0.071854,0.116324,0.137407
SP500,0.096611,0.150627,0.228182


In [5]:
random_sharpe = []
random_sortino = []
random_return = []
for i in range(1000):
    rand_stocks = random.sample(list(sp500_train.stock_sharpe.index), 30)
    random_sharpe.append(sp500_test.get_portf_sharpe(tick = rand_stocks))
    random_sortino.append(sp500_test.get_portf_sortino(tick = rand_stocks))
    random_return.append(sp500_test.get_portf_return(tick = rand_stocks))
baseline.loc['Random'] = [np.array(random_sharpe).mean(), np.array(random_sortino).mean(), np.array(random_return).mean()]
baseline

,Sharpe,Sortino,Returns
DJI,0.071854,0.116324,0.137407
SP500,0.096611,0.150627,0.228182
Random,0.084340,0.134432,0.222219


In [6]:
top30 = list(sp500_train.stock_sharpe.sort_values(by = 'Sharpe', ascending = False).iloc[:30].index)
baseline.loc['Top30'] = [sp500_test.get_portf_sharpe(tick = top30), 
                         sp500_test.get_portf_sortino(tick = top30), 
                         sp500_test.get_portf_return(tick = top30)]
baseline

,Sharpe,Sortino,Returns
DJI,0.071854,0.116324,0.137407
SP500,0.096611,0.150627,0.228182
Random,0.084340,0.134432,0.222219
Top30,0.122589,0.190844,0.264911


In [7]:
from scipy.stats import ttest_1samp

benchmark = dji_test.get_portf_sharpe()

# Perform one-sample t-test
t_statistic, p_value = ttest_1samp(np.array(random_sharpe), benchmark)

# Since ttest_1samp performs a two-tailed test, we need to adjust p-value for one-tailed test
p_value_one_tailed = p_value / 2 if t_statistic > 0 else 1 - (p_value / 2)

print(f"T-Statistic: {t_statistic}")
print(f"P-Value (one-tailed): {p_value_one_tailed}")

T-Statistic: 14.162593970834317
P-Value (one-tailed): 6.24154224491388e-42


To set the baselines, we are using the Dow Jones Index which is a stock market index that measures the stock performance of 30 prominent companies listed on stock exchanges in the United States. We also used the average of a 30-trial Monte Carlo Simulation of 30 random stock portfolios.

## Preprocessing

### Scale Data: Standardization

In [13]:
X = sp500_train.prices.dropna(axis = 1).T
scaler = TimeSeriesScalerMeanVariance()
X_scaled = scaler.fit_transform(X.to_numpy())

(454, 2518)

## Representative-Based Clustering (RBC): KMeans & KMedoids

### KMeans for 30 Clusters
For comparison to the DJI, we set our preset our cluster to 30 and pick the best performing stock in each cluster based on Sharpe ratio from 2013 to 2023. Each clsuter is supposed to represent differing levels of returns. We are diversifying our portfolio by taking one from each.



In [17]:
km = TimeSeriesKMeans(n_clusters=30, verbose=False, random_state=42)
clusters = km.fit_predict(X_scaled)

In [27]:
df = pd.DataFrame(clusters, index=X.index, columns=['Cluster'])
df = pd.concat([df, sp500_train.stock_sharpe], axis=1)
portfolio = df.sort_values(by=['Cluster', 'Sharpe'], ascending=[True, False])
portfolio = list(portfolio.groupby('Cluster').head(1).index)

# Compute Sharpe ratio of the portfolio
sharpe = sp500_test.get_portf_sharpe(tick = portfolio)
sortino = sp500_test.get_portf_sortino(tick = portfolio)
returns = sp500_test.get_portf_return(tick = portfolio)
sharpe, sortino, returns

(0.09129949818695339, 0.1430529612054047, 0.21470590184464983)

In [39]:
distance_matrix.shape

(2518, 2518)

In [ ]:
ts_X = X.to_numpy()[:, :, None]
distance_matrix = cdist_dtw(ts_X)
distance_df = pd.DataFrame(distance_matrix, index=X.index, columns=X.index)
print(distance_df)

In [29]:
dba_km = TimeSeriesKMeans(n_clusters=30, metric = 'dtw', verbose=False, max_iter =20, random_state=42)
clusters = dba_km.fit_predict(X_scaled)

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(clusters, index=X.index, columns=['Cluster'])
df = pd.concat([df, sp500_train.stock_sharpe], axis=1)
portfolio = df.sort_values(by=['Cluster', 'Sharpe'], ascending=[True, False])
portfolio = list(portfolio.groupby('Cluster').head(1).index)

# Compute Sharpe ratio of the portfolio
sharpe = sp500_test.get_portf_sharpe(tick = portfolio)
sortino = sp500_test.get_portf_sortino(tick = portfolio)
returns = sp500_test.get_portf_return(tick = portfolio)
sharpe, sortino, returns